<a href="https://colab.research.google.com/github/nmills/climate-utility-functions/blob/main/Climate_Analysis_Utility_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploration of Ambient Conditions


In [ ]:
import math
import numpy as np

## Feels Like Temperature

Calculating the apparent "feels like" temperature

Formula adopted from the work done by [Steadman et. al. (1994)](http://www.bom.gov.au/jshess/docs/1994/steadman.pdf) 

This equation calculates **apparent temperature (AT)** or **heat index**, which is a measure of how hot it feels to the human body when relative humidity and wind speed are factored in along with the actual air temperature. Let's break down each component of the equation:

\[
AT = Ta + 0.33 \times e - 0.70 \times ws - 4.00
\]

Where:

1. **\( AT \)**: This represents the **apparent temperature** or **heat index**. It is the temperature perceived by humans, taking into account the combined effects of air temperature, humidity, and wind speed.

2. **\( Ta \)**: This is the **air temperature** in degrees Celsius (°C). It is the actual temperature of the air.

3. **\( 0.33 \times e \)**: This term accounts for the effect of **vapor pressure (e)** on the apparent temperature. 
   - **\( e \)**: The actual vapor pressure, usually measured in hectopascals (hPa) or millibars (mb), which reflects the amount of moisture in the air. It is calculated using the relative humidity and temperature, as explained in the previous breakdown.
   - **0.33**: This coefficient is an empirical value that represents the effect of humidity on the apparent temperature. Higher humidity increases the apparent temperature, making it feel warmer.

4. **\( -0.70 \times ws \)**: This term represents the effect of **wind speed (ws)** on the apparent temperature.
   - **\( ws \)**: Wind speed in meters per second (m/s). 
   - **−0.70**: This coefficient is an empirical value that quantifies how wind reduces the apparent temperature. Wind has a cooling effect on the body, so as wind speed increases, the apparent temperature decreases.

5. **−4.00**: This is a constant that adjusts the overall formula to better match human perception of temperature under various conditions. It represents the baseline effect of the body losing heat to the environment, independent of other factors.

### Summary:

- **Apparent Temperature (AT)**: This equation combines the effects of actual air temperature, humidity (through vapor pressure \( e \)), and wind speed to estimate how hot it feels to humans. 
- **Positive Contributions:**
  - The actual air temperature \( Ta \).
  - The effect of humidity, which makes it feel hotter (0.33 × e).
- **Negative Contributions:**
  - The effect of wind speed, which cools the body (−0.70 × ws).
  - The constant adjustment (−4.00) to align the equation with human perception.

This formula is used in meteorology and human comfort studies to better understand how environmental factors affect perceived temperature, which is used for public health and safety during extreme weather conditions.

This is the calculation of **vapor pressure** in meteorology.

\[
e = \frac{rh}{100} \times 6.105 \times \exp \left( \frac{17.27 \times Ta}{237.7 + Ta} \right)
\]

Where:

1. **\( e \)**: This represents the actual vapor pressure in the atmosphere. It's typically measured in hectopascals (hPa) or millibars (mb).

2. **\( rh \)**: This stands for relative humidity, which is the percentage of moisture in the air compared to the maximum amount of moisture the air can hold at that temperature. It is given as a percentage.

3. **\( \frac{rh}{100} \)**: This converts the relative humidity from a percentage to a decimal. For example, if the relative humidity is 50%, this term would become 0.5.

4. **6.105**: This is a constant that is used in the empirical equation to estimate the saturation vapor pressure. The value might slightly vary depending on the specific formulation, but it is generally close to this value in equations related to meteorology.

5. **\( \exp \)**: This is the exponential function, often written as \( e^x \) where \( e \) is the base of the natural logarithm (approximately equal to 2.71828). In this context, it is used to calculate the saturation vapor pressure based on temperature.

6. **\( Ta \)**: This represents the air temperature in degrees Celsius (°C).

7. **\( \frac{17.27 \times Ta}{237.7 + Ta} \)**: This is part of the empirical formula for calculating the saturation vapor pressure at a given air temperature. The constants 17.27 and 237.7 are derived from the Clausius-Clapeyron equation and are commonly used in such formulations to estimate vapor pressure.

### Therefore:

- **Vapor Pressure Calculation:** The equation calculates the actual vapor pressure \( e \) in the air by taking into account the relative humidity and the saturation vapor pressure (which is a function of the temperature \( Ta \)). 

- **Saturation Vapor Pressure:** The term \( 6.105 \times \exp \left( \frac{17.27 \times Ta}{237.7 + Ta} \right) \) estimates the saturation vapor pressure at the given temperature.

- **Actual Vapor Pressure:** The final vapor pressure \( e \) is then obtained by multiplying the saturation vapor pressure by the relative humidity (as a decimal).

This equation is useful in various meteorological calculations, including determining dew point, humidity, and other related atmospheric properties.


In [ ]:
def calculate_water_vapour_pressure(relative_humidity, dry_bulb_temperature):
    """
    Calculate the value for water vapour pressure.
    e = rh / 100 × 6.105 × exp ( 17.27 × Ta / ( 237.7 + Ta ) )

    :parameters
    ----------
    relative_humidity:

    dry_bulb_temperature:

    :return: e or water vapour pressure
    """

    return (relative_humidity / 100) * 6.105 * math.exp((17.27 * dry_bulb_temperature) / (237.7 + dry_bulb_temperature))

In [ ]:
def calculate_apparent_temperature(dry_bulb_temperature, relative_humidity, wind_speed):
    """
    Including the effects of temperature, humidity, and wind:
    AT = Ta + 0.33×e − 0.70×ws − 4.00

    :parameters
    ----------
    dry_bulb_temperature:

    relative_humidity:

    wind_speed:

    :return:
    """

    return (dry_bulb_temperature + 0.33 * calculate_water_vapour_pressure(relative_humidity, dry_bulb_temperature)) \
           - (0.7 * (wind_speed / 3.6)) \
           - 4

In [ ]:
print(calculate_apparent_temperature(10.5, 69.0, 9.0))
print(calculate_apparent_temperature(7.7, 91.0, 9.0))

7.636319392877429
5.101950662573325


The function below will bring our Apparent Temperature calculation functions together in the entry point of a Google Cloud Function. It assumes that the parameters that have been passed are dry_bulb_temperature, relative_humidity, wind_speed.

Once deployed you can use the following line in the cloud console to call the function

`gcloud functions call get-feels-like-temp --data '{ "dry_bulb_temperature":"10.5", "relative_humidity":"69.0", "wind_speed":"9.0" }'

In [ ]:
def cal_app_tmp(request):
    """Responds to any HTTP request.
    Args:
        request (flask.Request): HTTP request object.
    Returns:
        The response text or any set of values that can be turned into a
        Response object using
        `make_response <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>`.
    """
    request_json = request.get_json()

    if not request_json:
        return {"message": "Request parameters corrupted."}, 400

    try:

        dry_bulb_temperature = float(request_json['dry_bulb_temperature'])
        relative_humidity = float(request_json['relative_humidity'])
        wind_speed = float(request_json['wind_speed'])

    except KeyError as err:
        return {"message": "No request parameters available"}, 422

    return str(calculate_apparent_temperature(dry_bulb_temperature, relative_humidity, wind_speed))